# PROJET 7

In [2]:
import mlflow
# demarrage d'une "expérience" 
mlflow.set_experiment("Risk Scoring")

<Experiment: artifact_location='file:///Users/danongohou/Desktop/P7%20Implementer%20un%20modele%20de%20scoring/mlruns/854010955790355566', creation_time=1685533245664, experiment_id='854010955790355566', last_update_time=1685533245664, lifecycle_stage='active', name='Risk Scoring', tags={}>

In [8]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from dataprep import eda
import qgrid
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import joblib 
from numba import njit
from numba import generated_jit
from numba import types

%matplotlib inline 

import gc
import time
from contextlib import contextmanager
# File system manangement
import os
import os.path
from os import path 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.datasets import make_classification

from collections import Counter

from imblearn.over_sampling import SMOTE 
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import TomekLinks

import scipy.stats as stats

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
import shap
import pickle
import joblib

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

bleu = (70 / 255, 130 / 255, 180 / 255)
pd.set_option('display.max_row', 219)
pd.set_option('display.max_column', 243)
pd.set_option('display.max_colwidth', 50)

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

In [4]:
mlflow.start_run(run_name='Features Importance') 

<ActiveRun: >

## LGBM

In [5]:
# Charger les descripteurs ORB extraits dans la question précédente
with open('./data/data_for_modelling_final.pickle', 'rb') as f:
    data_for_modelling = pickle.load(f)

In [17]:
with open('./Models/model_lgbm.pkl', 'rb') as f:
    model_lgbm = pickle.load(f)

In [11]:
X = data_for_modelling.copy()

In [12]:
y = X['TARGET']

In [13]:
X.drop(columns=['TARGET','SK_ID_CURR'],inplace=True)

In [14]:
X_test,X_train,y_test,y_train=train_test_split(X,y,test_size=0.20)

# StandardScaler
# Fit on the training data
scaler = StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [15]:
lgbm=LGBMClassifier(learning_rate=0.07, 
                    max_depth=3, 
                    n_estimators=200, 
                    num_leaves= 8)

kfold = KFold(n_splits=5, shuffle=True, random_state=123)
scoring = {'accuracy': 'accuracy',
           'precision_weighted': 'precision_weighted',
           'recall_weighted': 'recall_weighted',
           'f1_weighted': 'f1_weighted',
           'roc_auc': 'roc_auc',
           'fbeta': make_scorer(fbeta_score, average='weighted', beta=3)       
           # CREER LE SCORE METIER 
           }
# CREER LE SCORE METIER 

In [18]:
model = lgbm
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.28)
steps = [('over', over), ('under', under), ('model', model)]
pipeline = Pipeline(steps=steps)

kfold = KFold(n_splits=5, shuffle=True, random_state=123)
cv_results = cross_validate(pipeline, X_train, y_train,
                            cv=kfold, scoring=scoring)
model_lgbm = model.fit(X_train, y_train)
y_pred = model_lgbm.predict(X_test)



In [19]:
df = pd.DataFrame(cv_results)
df_smote_rand=pd.DataFrame(df.mean(axis=0)).transpose()
df_smote_rand

,fit_time,score_time,test_accuracy,test_precision_weighted,test_recall_weighted,test_f1_weighted,test_roc_auc,test_fbeta
0,1.119043,0.05964,0.900359,0.878912,0.900359,0.887335,0.758826,0.897519


## Explicabilité Globale

In [ ]:
shap.initjs()
feats = [f for f in data_for_modelling.columns if f not in ['TARGET', 'SK_ID_CURR']]
ex = shap.TreeExplainer(model_lgbm, X_train, feature_names = feats)
shap_values = ex.shap_values(X_test)
shap.summary_plot(shap_values, X_test, feature_names=X.columns, max_display=40)


## Explicabilité locale

In [ ]:
shap.bar_plot(explainer.shap_values(X_test[10]), feature_names=np.array(feats), max_display=10)

In [21]:
mlflow.end_run()

In [ ]:
!mlflow ui 

[2023-05-31 14:57:35 +0200] [4911] [INFO] Starting gunicorn 20.1.0
[2023-05-31 14:57:35 +0200] [4911] [INFO] Listening at: http://127.0.0.1:5000 (4911)
[2023-05-31 14:57:35 +0200] [4911] [INFO] Using worker: sync
[2023-05-31 14:57:35 +0200] [4912] [INFO] Booting worker with pid: 4912
[2023-05-31 14:57:35 +0200] [4913] [INFO] Booting worker with pid: 4913
[2023-05-31 14:57:35 +0200] [4914] [INFO] Booting worker with pid: 4914
[2023-05-31 14:57:35 +0200] [4915] [INFO] Booting worker with pid: 4915
[2023-05-31 15:37:06 +0200] [4911] [CRITICAL] WORKER TIMEOUT (pid:4912)
[2023-05-31 15:37:06 +0200] [4911] [CRITICAL] WORKER TIMEOUT (pid:4913)
[2023-05-31 15:37:06 +0200] [4911] [CRITICAL] WORKER TIMEOUT (pid:4914)
[2023-05-31 15:37:06 +0200] [4911] [CRITICAL] WORKER TIMEOUT (pid:4915)
[2023-05-31 15:37:06 +0200] [4912] [INFO] Worker exiting (pid: 4912)
[2023-05-31 15:37:06 +0200] [4913] [INFO] Worker exiting (pid: 4913)
[2023-05-31 15:37:06 +0200] [4914] [INFO] Worker exiting (pid: 4914)
[202